# RAG Testing

In [31]:
import langchain_community.llms
from langchain_ollama import ChatOllama
print(langchain_community.llms.__file__)
from langchain_community.embeddings import OllamaEmbeddings

/home/tmoleary/school/cs542-adventure/env/lib/python3.13/site-packages/langchain_community/llms/__init__.py


In [50]:
model = ChatOllama(model='gpt-oss', max_retry=5)
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

In [51]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
from typing import Iterable
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# docs should already have metadata attached. Document objects have a 'metadata' field that is just a dictionary
# (https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document.metadata)
def insert_into_vector_store(docs: Iterable[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # chunk size (characters)
        chunk_overlap=100,  # chunk overlap (characters)
        add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(docs)

    document_ids = vector_store.add_documents(documents=all_splits)

    print(f"Split docs into {len(all_splits)} sub-documents.")


In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def remember(query: str):
    """Remember relevant past events during gameplay in order to inform the next step"""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [54]:
import jericho
print(jericho.__file__)
import sys
print(sys.path)

GAMES_DIR = "z-machine-games-master/jericho-game-suite"
game = 'zork1.z5'
env = jericho.FrotzEnv(f"{GAMES_DIR}/{game}")

/home/tmoleary/school/cs542-adventure/env/lib/python3.13/site-packages/jericho/__init__.py
['/usr/lib/python313.zip', '/usr/lib/python3.13', '/usr/lib/python3.13/lib-dynload', '', '/home/tmoleary/school/cs542-adventure/env/lib/python3.13/site-packages']


In [ ]:
import time
from langchain.agents import create_agent
from langchain_core.documents import Document

def agent(out_file=sys.stdout):
    original_stdout = sys.stdout
    sys.stdout = out_file

    print("Playing game. 'q' to quit")

    game_response, info = env.reset()

    done = False

    @tool(response_format="content")
    def do_game_action(action: str) -> str:
        """Perform an action in the active text adventure game and see the result"""
        """
        Args:
          action: game action string

        Returns:
          The game's response after performing the action
        """
        nonlocal done, game_response, info
        game_response, reward, done, info = env.step(action)
        if done:
            game_response += '\nYou have finished the game!'
        return game_response
    
    @tool(response_format="content")
    def view_possible_actions() -> str:
        """View a list of the actions that can be performed in the game's current state"""
        """
        Returns:
          String containg actions separated by commas
        """
        return ', '.join(env.get_valid_actions())
    
    # def view_walkthrough():
    #     """View the full game walkthrough as a list of actions"""
    #     """
    #     Returns:
    #       String containing actions separated by newlines
    #     """
    #     return env.get_walkthrough()
    


    tools = [remember, do_game_action, view_possible_actions]
    system_prompt = (
        f"You are playing {game}, an interactive fiction game. You must analyze the scenario the game presents to you and choose an action that will make progress. Your goal is to finish the game\n"
        "You have access to a tool that allows you to remember past events that have occured in your current playthrough that are relevant to your situation. "
        "Use the tool to help you decide on the next action to take in-game "
    )
    agent = create_agent(model, tools, system_prompt=system_prompt)

    generate_times = []
    prev_score = 0

    while not done:
        query = (
            "Think critically. Finish the game. Here is your current scenario:\n"
            f"{game_response}"
        )

        start = time.time()
        for event in agent.stream(
            {"messages": [{"role": "user", "content": query}]},
            stream_mode="values",
        ):
            generate_times.append(time.time() - start)

            last_message = event["messages"][-1]
            last_message.pretty_print()
            
            document = Document(
                page_content=last_message.content, metadata={"move": info['moves']}
            )
            insert_into_vector_store([document])

            start = time.time()
            
            if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
                print('Score changed! New score:', info['score'])
            prev_score = info['score']
            
    print('Scored', info['score'], 'out of', env.get_max_score())

    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [56]:
agent()

Playing game. 'q' to quit
================================ Human Message =================================

Think critically. Finish the game. Here is your current scenario:
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


Split docs into 5 sub-documents.
================================== Ai Message ==================================
Tool Calls:
  do_game_action (d8c70f9a-6377-455c-98c0-1a162a47217b)
 Call ID: d8c70f9a-6377-455c-98c0-1a162a47217b
  Args:
    action: north
Split docs into 0 sub-documents.
================================= Tool Message =================================
Name: do_game_action

North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow path winds

ResponseError: error parsing tool call: raw='We need to progress to finish Zork. We are at clearing with pile of leaves. Need to find key to open something? Classic Zork opening. The key is in a tree trunk, etc. Leaves maybe used to open? Maybe need to take leaves? They are there. But "take leaves" gave error earlier: "You can't see any leaves here!" That was when we were at forest path? Wait earlier we attempted "take leaves" in forest path. Now in clearing we see leaves. Should be able to "take leaves". Let's try.{"action":"take leaves"}', err=invalid character 'W' looking for beginning of value (status code: 500)